In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import h5py
import scipy.signal as ss
import LFPy
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import example_network_parameters as params
import example_network_methods as methods
from plotting import draw_lineplot, remove_axis_junk, annotate_subplot
import plotting

In [ ]:
plt.rcParams.update(plotting.rcParams)
golden_ratio = plotting.golden_ratio
figwidth = plotting.figwidth

In [ ]:
# downsample signals for plots to resoultion dt * decimate_ratio
decimate_ratio = 4

In [ ]:
TRANSIENT = 2000
T = [2000, 2400]
tstop = params.networkParameters['tstop']
dt = params.networkParameters['dt']

In [ ]:
# ss.welch/plt.mlab.psd/csd settings
Fs = 1000 / dt
NFFT = 1024 * 2
noverlap = 768 * 2
detrend = False
freqs_cutoff = 300.  # (Hz) ignore freqs above this in spectra

In [ ]:
# Dataset
OUTPUTPATH = os.path.join('output', 'adb947bfb931a5a8d09ad078a6d256b0')

In [ ]:
# Extract median soma voltages from actual network simulation and
# assume this value corresponds to Vrest.
Vrest = {}
with h5py.File(os.path.join(OUTPUTPATH, 'somav.h5'
                            ), 'r') as f:
    for Y in params.population_names:
        Vrest[Y] = np.median(f[Y][()][:, TRANSIENT:])
Vrest

In [ ]:
fig = plt.figure(figsize=(figwidth, figwidth))
fig.subplots_adjust(wspace=0.7, hspace=0.3)
gs = GridSpec(5, 21)

####
# POPULATIONS
####
ax = fig.add_subplot(gs[:, :3])
remove_axis_junk(ax)

# network populations
X = ['E', 'I']
N_X = [128, 16]
m = ['BallAndSticks_E.hoc', 'BallAndSticks_I.hoc']

# E and I colors
colors = ['tab:blue', 'tab:red']

# laminar probe
x_e = params.electrodeParameters['x']
z_e = params.electrodeParameters['z']
ax.plot(x_e, z_e, 'ko', label='el.')

for i, (x_i, z_i) in enumerate(zip(x_e, z_e)):
    if (i < 3) or (i > 10):
        ax.text(x_i-10, z_i, f'ch.{i+1}', va='center', ha='right')

with h5py.File(os.path.join(OUTPUTPATH, 'cell_positions_and_rotations.h5'), 'r') as f:
    for i, (X_, N_X_, m_) in enumerate(zip(X, N_X, m)):
        for j in range(N_X_):
            cellParameters = dict(
                morphology=m_,
                templatefile='BallAndSticksTemplate.hoc',
                templatename='BallAndSticksTemplate',
                templateargs=None,
                delete_sections=True,
            )
            cell = LFPy.TemplateCell(**cellParameters)
            cell.set_pos(f[X_][j]['x'], f[X_][j]['y'], f[X_][j]['z'])

            # show morphology
            zips = []
            for x, z in cell.get_idx_polygons(projection=('x', 'z')):
                zips.append(list(zip(x, z)))

            polycol = PolyCollection(zips,
                                    edgecolors='w',
                                    facecolors=colors[i],
                                    linewidths=0.25,
                                    zorder=np.random.rand()-0.5,
                                    label=X_ if j == 0 else '__nolabel__')
            ax.add_collection(polycol)
    
ax.axis('equal')
ax.set_xlabel('$x$ (µm)')
ax.set_ylabel('$z$ (µm)')
ax.legend()
annotate_subplot(ax, ncols=7, nrows=1, letter='A', linear_offset=0.02)



######
# Cell positions
######
ax1 = fig.add_subplot(gs[:, 3:6], sharey=ax)
remove_axis_junk(ax1)

bins = np.arange(-200, 1001, 20)

with h5py.File(os.path.join(OUTPUTPATH, 'cell_positions_and_rotations.h5'), 'r') as f:
    for i, X_ in enumerate(X):
        ax1.hist(f[X_][:]['z'], bins=bins, 
                 histtype='step',
                 orientation='horizontal',
                 color=colors[i],
                 label='X={}'.format(X_))
ax1.legend()
ax1.set_xlabel('#')
plt.setp(ax1.get_yticklabels(), visible=False)
annotate_subplot(ax1, ncols=7, nrows=1, letter='B', linear_offset=0.02)


######
# synapse counts
#####
ax2 = fig.add_subplot(gs[:, 6:9], sharey=ax)
remove_axis_junk(ax2)

with h5py.File(os.path.join(OUTPUTPATH, 'synapse_connections.h5'), 'r') as f:
    bins = np.arange(-200, 1001, 20)
    data = []
    labels = []
    for x in X:
        for y in X:
            data.append(f['{}:{}'.format(x, y)]['z'])   # notation PRE:POST
            labels.append('$Y$={};$X$={}'.format(y, x))
    ax2.hist(data, 
            bins=bins, 
            # labels=labels, 
            histtype='step',
            orientation='horizontal',
            alpha=1,
            color=[f'C{k + 4}' for k in range(len(X)**2)],
            label=labels)
ax2.legend()
ax2.set_xlabel('#')
plt.setp(ax2.get_yticklabels(), visible=False)
annotate_subplot(ax2, ncols=7, nrows=1, letter='C', linear_offset=0.02)



#####
# SOMATIC VOLTAGES
#####
ax2 = fig.add_subplot(gs[0, 10:15])
remove_axis_junk(ax2)
ax = fig.add_subplot(gs[1, 10:15])
remove_axis_junk(ax)

'''
divider = make_axes_locatable(ax)
ax2 = divider.new_vertical(size="100%", pad=0.1)
fig.add_axes(ax2)
'''



for i, (Y, gid0, a) in enumerate(zip(params.population_names, [0, params.population_sizes[0]], [ax2, ax])):
    with h5py.File(os.path.join(OUTPUTPATH, 'somav.h5'), 'r') as f:
        data = f[Y][:10, ]
    draw_lineplot(a,
                  data,
                  dt=1,
                  T=T,
                  scaling_factor=1.,
                  vlimround=None,
                  label=r'$\overline{V}_\mathrm{m}^\mathrm{\langle ' + f'{Y}' + 
                      r' \rangle}=' + f'{np.round(Vrest[Y], decimals=1)}' + f'$ mV',
                  scalebar=True,
                  unit='mV',
                  ylabels=True,
                  color=colors[i],
                  ztransform=True
                  )
    yticklabels = np.arange(gid0, gid0 + data.shape[0]).astype(object)
    yticklabels[1::2] = ''
    a.set_yticklabels(yticklabels)
    a.set_ylabel('gid')
    a.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left", borderaxespad=0)
    plt.setp(a.get_xticklabels(), visible=False)
    a.set_xlabel('')

annotate_subplot(ax2, ncols=7, nrows=5, letter='D', linear_offset=0.02)


####
# SPIKE RASTER
####
ax = fig.add_subplot(gs[2, 10:15])
remove_axis_junk(ax)

with h5py.File(os.path.join(OUTPUTPATH, 'spikes.h5'), 'r') as f:
    for i, y in enumerate(X):
        times = []
        gids = []

        for g, t in zip(f[y]['gids'], f[y]['times']):
            times = np.r_[times, t]
            gids = np.r_[gids, np.zeros(t.size) + g]

        gids = gids[times >= TRANSIENT]
        times = times[times >= TRANSIENT]

        ii = (times >= T[0]) & (times <= T[1])
        ax.plot(times[ii], gids[ii], '.',
                mfc=colors[i],
                mec='none',
                ms=2,
                label=r'$\langle \nu_\mathrm{%s} \rangle =%.1f$ s$^{-1}$' % (
                    y, times.size / f[y]['gids'].size /
                    (tstop - TRANSIENT) * 1000))
ax.legend(loc=1, markerscale=5)
ax.axis('tight')
ax.set_xticklabels([])
ax.set_ylabel('gid')
ax.invert_yaxis()
annotate_subplot(ax, ncols=7, nrows=5 / 3, letter='E', linear_offset=0.02)


#####
# Rates
####
ax = fig.add_subplot(gs[3, 10:15])
remove_axis_junk(ax)

nu_X, bin_edges = methods.compute_nu_X(os.path.join(OUTPUTPATH, 'spikes.h5'), X, T, Delta_t=1)
for i, X_i in enumerate(X):
    ax.step(bin_edges[:-1], nu_X[X_i], 
            color=colors[i], 
            label='$\\nu_{%s}$' % X_i)

ax.axis(ax.axis('tight'))
ax.legend(loc='upper right')
ax.set_xlabel('$t$ (ms)')
ax.set_ylabel('#')
annotate_subplot(ax, ncols=7, nrows=5, letter='F', linear_offset=0.02)


#### 
# Rate spectra
####
ax = fig.add_subplot(gs[4, 10:15])
remove_axis_junk(ax)
nu_X, bin_edges = methods.compute_nu_X(os.path.join(OUTPUTPATH, 'spikes.h5'), X, 
                                       T=(TRANSIENT, tstop), Delta_t=dt)
for i, (X_i) in enumerate(X):
    nu = nu_X[X_i] / (dt / 1000)  # spikes per s
    freqs, Pxx = ss.welch(nu, fs=Fs, nperseg=NFFT, noverlap=noverlap, detrend=detrend)
    ax.semilogy(freqs[freqs <= freqs_cutoff], Pxx[freqs <= freqs_cutoff], 
                color=colors[i],
                label='$\\nu_{%s}$' % X_i)

ax.axis(ax.axis('tight'))
ax.legend(loc='upper right')
ax.set_xlabel('$f$ (Hz)', labelpad=0)
ax.set_ylabel(r'$P_{\nu_' + f'{"X"}' + r'\nu_' + f'{"X"}' + r'}$ (s$^{-2}$/Hz)', labelpad=0)
annotate_subplot(ax, ncols=7, nrows=5, letter='G', linear_offset=0.02)

#### 
# extracellular potentials
#### 
fname = 'RecExtElectrode.h5'
unit = 'mv'
vlimround = 2**-1

ax = fig.add_subplot(gs[:-1, 16:])

with h5py.File(os.path.join(OUTPUTPATH, fname),
               'r') as f:
    data = f['data'][()]
    
    label = ''
    draw_lineplot(ax,
                  ss.decimate(data['imem'], 
                              q=decimate_ratio,
                              zero_phase=True),
                  dt=dt * decimate_ratio,
                  T=T,
                  scaling_factor=1.,
                  vlimround=vlimround,
                  label=label,
                  scalebar=True,
                  unit=unit,
                  ylabels=True,
                  color='k',
                  ztransform=True
                  )
    ax.set_title(label)
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_xlabel('')
ax.axis('tight')
annotate_subplot(ax, ncols=7, nrows=5 / 4, letter='H', linear_offset=0.02)


#### 
# current dipole moment
#### 
fname = 'CurrentDipoleMoment.h5'
scaling = 1E-4  # nAum --> nAcm unit conversion
unit = 'nAcm'
vlimround = 2**4

ax = fig.add_subplot(gs[-1, 16:])

with h5py.File(os.path.join(OUTPUTPATH, fname),
               'r') as f:
    data = f['data'][()]
    
    label = ''
    draw_lineplot(ax,
                  ss.decimate(data['imem'], 
                              q=decimate_ratio,
                              zero_phase=True) * scaling,
                  dt=dt * decimate_ratio,
                  T=T,
                  scaling_factor=1.,
                  vlimround=vlimround,
                  label=label,
                  scalebar=True,
                  unit=unit,
                  ylabels=True,
                  color='k',
                  ztransform=True
                  )
    ax.set_title(label)
    ax.set_ylabel('')
    ax.set_yticklabels(['$P_x$', '$P_y$', '$P_z$'])
ax.axis('tight')
annotate_subplot(ax, ncols=7, nrows=5, letter='I', linear_offset=0.02)

if not os.path.isdir('figures'):
    os.mkdir('figures')
fig.savefig(os.path.join('figures', 'figure03.pdf'), bbox_inches='tight', pad_inches=0)